In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to the folder you want to access
folder_path = "/content/drive/My Drive/Machine_Learning_24/Final_project/predict-energy-behavior-of-prosumers/"

# List the content of the folder
!ls "$folder_path"

 client.csv
'Copy of Predict Energy Behavior of Prosumers.gslides'
 county_id_to_name_map.json
 data_merger.ipynb
 electricity_prices.csv
 enefit
 example_test_files
 final_project_LBM_tuner_.ipynb
 final_project_theo_GRU.ipynb
 final_project_timesieres_transformer.ipynb
 forecast_weather.csv
 gas_prices.csv
 Graph_animation
 Graph_animation2
 Graph_animation3
 historical_weather.csv
'Predict Energy Behavior of Prosumers.gslides'
 public_timeseries_testing_util.py
 timesieries_transformer
 train.csv
 train_full.csv
 weather_station_to_county_mapping.csv


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [ ]:


# !pip install verstack
# from verstack import LGBMTuner

In [ ]:

def generate_featuers(
    df_data,
    df_client,
    df_gas_prices,
    df_electricity_prices,
    df_forecast_weather,
    df_historical_weather,
    df_weather_station_to_county_mapping,
    train_start_timestep = '2021-09-01 11:00:00',
    gas_end_date = '2022-01-10 23:00:00'

    ):

    df_weather_station_to_county_mapping = df_weather_station_to_county_mapping[df_weather_station_to_county_mapping.notnull().all(axis=1)].sort_values(by="county")
    result_dict = dict(zip(zip(round(df_weather_station_to_county_mapping['latitude'],1), round(df_weather_station_to_county_mapping['longitude'],1)), df_weather_station_to_county_mapping['county']))

    df_historical_weather = df_historical_weather[df_historical_weather['datetime'] >= train_start_timestep]


    #Merge df_data and df_client
    df_client_try = df_client.copy(deep=True)
    df_client_try['date'] = pd.to_datetime(df_client_try['date'])
    df_client_try['datetime'] = df_client_try['date'].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_client_try = df_client_try.explode('datetime')
    df_client_try = df_client_try.drop(['date','data_block_id'], axis=1)

    df_data['datetime'] = pd.to_datetime(df_data['datetime'])
    df_data = df_data.merge(df_client_try, on=['county','product_type','is_business','datetime'], how='left')

    endDate = df_client_try['datetime'].max()
    df_data = df_data[df_data['datetime'] <= endDate]


    #Merge df_data and df_gas_prices
    df_gas_try = df_gas_prices.copy(deep=True)
    df_gas_try['forecast_date'] = pd.to_datetime(df_gas_try['forecast_date'])
    df_gas_try['datetime'] = df_gas_try['forecast_date'].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_gas_try = df_gas_try.explode('datetime')
    df_gas_try = df_gas_try.drop(['forecast_date' , 'origin_date' , 'data_block_id'], axis=1)
    gas_end_date = df_gas_try['datetime'].max()
    df_data = df_data[df_data['datetime'] <= gas_end_date]
    df_data = df_data.merge(df_gas_try, on=['datetime'], how='left')

    #Merge df_data and df_electricity_prices
    df_electricity_prices_try = df_electricity_prices.copy(deep=True)
    df_electricity_prices_try.drop(columns=["origin_date" , 'data_block_id'], inplace=True)
    df_electricity_prices_try['forecast_date'] = pd.to_datetime(df_electricity_prices_try['forecast_date'])
    df_electricity_prices_try.rename(columns={"forecast_date": "datetime"}, inplace=True)
    df_data = df_data.merge(df_electricity_prices_try, on=['datetime'], how='left')

    #Merge df_data and df_forecast_weather
    df_forecast_weather_copy = df_forecast_weather.copy(deep=True)
    df_forecast_weather_copy['county'] = [result_dict.get((x, y), -1) for x, y in zip(df_forecast_weather_copy['latitude'], df_forecast_weather_copy['longitude'])]
    df_forecast_weather_copy = df_forecast_weather_copy[df_forecast_weather_copy['county']!=-1]
    df_forecast_weather_copy['origin_datetime'] = pd.to_datetime(df_forecast_weather_copy['origin_datetime'])
    df_forecast_weather_copy['origin_datetime'] = pd.to_datetime(df_forecast_weather_copy['origin_datetime'].dt.date.astype(str) + ' 02:00:00')
    df_forecast_weather_copy['forecast_datetime'] = df_forecast_weather_copy['origin_datetime'] + pd.to_timedelta(df_forecast_weather_copy['hours_ahead'], unit='h')
    df_forecast_weather_copy = df_forecast_weather_copy.drop('origin_datetime',axis=1)
    df_forecast_weather_copy = df_forecast_weather_copy.sort_values(by=['latitude','longitude','forecast_datetime', 'hours_ahead'])
    df_forecast_weather_copy['cumcount'] = (df_forecast_weather_copy['hours_ahead']-1)//24+1

    columns_to_average = [col for col in df_forecast_weather_copy.columns if col not in ['latitude', 'longitude', 'hours_ahead' , 'forecast_datetime','cumcount']]
    agg_dict = {col: 'mean' for col in columns_to_average}
    agg_dict['cumcount'] = 'first'  # to preserve the cumcount value
    df_forecast_weather_copy = df_forecast_weather_copy.groupby(['county','forecast_datetime', 'cumcount']).agg(agg_dict)
    df_forecast_weather_copy=df_forecast_weather_copy.unstack(level=-1)
    df_forecast_weather_copy.columns = [f'{col[0]}_{col[1]}' for col in df_forecast_weather_copy.columns]
    df_forecast_weather_copy.reset_index(inplace=True)
    df_forecast_weather_copy.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)
    df_forecast_weather_copy = df_forecast_weather_copy.drop(['county_1','county_2','cumcount_1','cumcount_2'],axis=1)
    df_forecast_weather_copy.fillna(0, inplace=True)

    df_data = df_data.merge(df_forecast_weather_copy, on=['county','datetime'], how='left')


    #Merge df_data and df_historical_weather
    df_historical_weather_copy = df_historical_weather.copy(deep=True)
    df_historical_weather_copy['datetime'] = pd.to_datetime(df_historical_weather_copy['datetime'])
    df_historical_weather_copy['county'] = [result_dict.get((x, y), -1) for x, y in zip(df_historical_weather_copy['latitude'], df_historical_weather_copy['longitude'])]
    df_historical_weather_copy = df_historical_weather_copy[df_historical_weather_copy['county']!=-1]
    df_historical_weather_copy = df_historical_weather_copy.sort_values(by=['latitude','longitude','datetime'])
    columns_to_average = [col for col in df_historical_weather_copy.columns if col not in ['latitude', 'longitude', 'datetime','county','data_block_id']]
    agg_dict = {col: 'mean' for col in columns_to_average}
    df_historical_weather_copy = df_historical_weather_copy.groupby(['county','datetime']).agg(agg_dict)
    df_historical_weather_copy.reset_index(inplace=True)

    df_data = df_data.merge(df_historical_weather_copy, on=['county','datetime'], how='left')

    return df_data

In [ ]:
import pandas as pd

In [ ]:
DATA_DIR = folder_path
# Read CSVs and parse relevant date columns
df_data = pd.read_csv(DATA_DIR + "train.csv")
df_client = pd.read_csv(DATA_DIR + "client.csv")
df_historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
df_forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
df_electricity_prices = pd.read_csv(DATA_DIR + "electricity_prices.csv")
df_gas_prices = pd.read_csv(DATA_DIR + "gas_prices.csv")
df_weather_station_to_county_mapping = pd.read_csv(DATA_DIR + "weather_station_to_county_mapping.csv")

In [ ]:
combined_df = generate_featuers(
    df_data,
    df_client,
    df_gas_prices,
    df_electricity_prices,
    df_forecast_weather,
    df_historical_weather,
    df_weather_station_to_county_mapping)

In [ ]:
combined_df =combined_df.dropna()

In [ ]:
# Separate the dataset based on is_consumption column
#df_consumption_0 = combined_df[combined_df['is_consumption'] == 1]

In [ ]:
# target = df_consumption_0['target']
# # Drop all rows with any NaN values
# cleaned_df = target.dropna()

NameError: name 'df_consumption_0' is not defined

In [ ]:
combined_df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,...,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation
1342,0,0,1,391.054,0,2021-09-01 11:00:00,0,1342,0,108.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
1343,0,0,1,31.147,1,2021-09-01 11:00:00,0,1343,0,108.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
1344,0,0,2,86.409,0,2021-09-01 11:00:00,0,1344,1,17.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
1345,0,0,2,10.018,1,2021-09-01 11:00:00,0,1345,1,17.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
1346,0,0,3,3217.033,0,2021-09-01 11:00:00,0,1346,2,688.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012107,15,1,0,188.167,1,2023-05-29 23:00:00,635,2012107,64,15.0,...,1003.740000,21.200000,2.800000,16.400000,29.800000,1.772222,170.000000,0.000000,0.000000,0.000000
2012108,15,1,1,0.000,0,2023-05-29 23:00:00,635,2012108,59,20.0,...,1003.740000,21.200000,2.800000,16.400000,29.800000,1.772222,170.000000,0.000000,0.000000,0.000000
2012109,15,1,1,31.484,1,2023-05-29 23:00:00,635,2012109,59,20.0,...,1003.740000,21.200000,2.800000,16.400000,29.800000,1.772222,170.000000,0.000000,0.000000,0.000000
2012110,15,1,3,0.000,0,2023-05-29 23:00:00,635,2012110,60,55.0,...,1003.740000,21.200000,2.800000,16.400000,29.800000,1.772222,170.000000,0.000000,0.000000,0.000000


In [ ]:
import plotly.express as px
product_type = 3
is_business = 0
county = 0
is_consumption = 1

df_plot = combined_df[
    (combined_df['is_business'] == is_business)
    & (combined_df['is_consumption'] == is_consumption)
    & (combined_df['product_type'] == product_type)
    & (combined_df['county'] == county)
]
print(df_plot)

         county  is_business  product_type    target  is_consumption  \
1347          0            0             3   136.424               1   
1469          0            0             3   126.275               1   
1591          0            0             3   137.347               1   
1713          0            0             3   199.337               1   
1835          0            0             3   237.325               1   
...         ...          ...           ...       ...             ...   
2011467       0            0             3   826.332               1   
2011597       0            0             3  1094.914               1   
2011727       0            0             3  1473.528               1   
2011857       0            0             3  1555.478               1   
2011987       0            0             3  1390.501               1   

                   datetime  data_block_id   row_id  prediction_unit_id  \
1347    2021-09-01 11:00:00              0     1347         

In [ ]:
df_plot

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,...,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation
1347,0,0,3,136.424,1,2021-09-01 11:00:00,0,1347,2,688.0,...,1007.066667,27.166667,18.666667,16.666667,2.333333,6.291667,326.166667,466.166667,342.833333,123.333333
1469,0,0,3,126.275,1,2021-09-01 12:00:00,0,1469,2,688.0,...,1007.100000,29.000000,16.000000,24.333333,0.000000,6.268519,326.666667,496.166667,390.666667,105.500000
1591,0,0,3,137.347,1,2021-09-01 13:00:00,0,1591,2,688.0,...,1006.916667,37.666667,13.500000,42.166667,0.000000,6.532407,324.500000,508.000000,360.666667,147.333333
1713,0,0,3,199.337,1,2021-09-01 14:00:00,0,1713,2,688.0,...,1006.816667,41.666667,18.500000,41.500000,0.000000,6.828704,322.500000,503.500000,381.500000,122.000000
1835,0,0,3,237.325,1,2021-09-01 15:00:00,0,1835,2,688.0,...,1006.500000,35.000000,14.500000,36.666667,0.000000,6.912037,320.500000,418.500000,301.166667,117.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011467,0,0,3,826.332,1,2023-05-29 19:00:00,635,2011467,2,1515.0,...,1012.966667,51.000000,0.000000,58.333333,53.500000,4.402778,286.500000,229.666667,155.000000,74.666667
2011597,0,0,3,1094.914,1,2023-05-29 20:00:00,635,2011597,2,1515.0,...,1012.950000,50.666667,0.000000,70.333333,27.333333,3.800926,281.666667,108.333333,50.500000,57.833333
2011727,0,0,3,1473.528,1,2023-05-29 21:00:00,635,2011727,2,1515.0,...,1012.766667,59.000000,0.000000,82.666667,31.500000,3.490741,276.833333,27.000000,7.000000,20.000000
2011857,0,0,3,1555.478,1,2023-05-29 22:00:00,635,2011857,2,1515.0,...,1012.933333,59.666667,0.000000,82.500000,34.000000,3.421296,275.166667,0.000000,0.000000,0.000000


In [ ]:
df_plot['target']

1347        136.424
1469        126.275
1591        137.347
1713        199.337
1835        237.325
             ...   
2011467     826.332
2011597    1094.914
2011727    1473.528
2011857    1555.478
2011987    1390.501
Name: target, Length: 15247, dtype: float64

#Try other

In [ ]:
# papers for multivariant models
#https://github.com/andresC98/TSF_Transformers_TFM/blob/main/code_notebooks/%5BTFM%5D%20TFT%20Model%20-%20Traffic%20%26%20Electricity%20Benchmarks%20Tests.ipynb
#https://arxiv.org/pdf/1912.09363
#https://pytorch-forecasting.readthedocs.io/en/stable/_modules/pytorch_forecasting/models/temporal_fusion_transformer.html#TemporalFusionTransformer
#https://huggingface.co/blog/autoformer

In [ ]:
len(df_plot[:10000])
fig = px.line(
    df_plot,
    x="datetime",
    y="target",
    color='county',
    title=f'Target for is_busines={is_business}, is_consumption={is_consumption}, product_type={product_type}'
)
fig.show()

In [ ]:
df_plot['euros_per_mwh']

1347       109.75
1469       103.88
1591        98.00
1713        94.98
1835        95.06
            ...  
2011467     82.30
2011597     82.30
2011727     82.30
2011857     82.29
2011987     77.27
Name: euros_per_mwh, Length: 15247, dtype: float64

In [ ]:
target_and_euros_per_mwh = df_plot[['target', 'euros_per_mwh']]

In [ ]:
df_plot.dropna(subset=['target'], inplace=True)

<ipython-input-15-ee84b19c0a00>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_plot['target']

1347        136.424
1469        126.275
1591        137.347
1713        199.337
1835        237.325
             ...   
2011467     826.332
2011597    1094.914
2011727    1473.528
2011857    1555.478
2011987    1390.501
Name: target, Length: 15247, dtype: float64

In [ ]:
# Assuming df_plot is your DataFrame
target_and_euros_per_mwh.dropna(subset=['target'], inplace=True)

<ipython-input-29-da709496a1b2>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
target_and_euros_per_mwh

,target,euros_per_mwh
5,656.859,92.51
127,595.498,88.90
249,598.450,87.35
371,622.824,86.88
493,607.308,88.43
...,...,...
2011467,826.332,82.30
2011597,1094.914,82.30
2011727,1473.528,82.30
2011857,1555.478,82.29


# Try to make a multimodel


In [ ]:
folder_path_graph = "/content/drive/My Drive/Machine_Learning_24/Final_project/predict-energy-behavior-of-prosumers/Graph_animation3/"
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module='torch.nn.modules.transformer')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.optim.lr_scheduler')

import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot as plt

torch.manual_seed(2)
np.random.seed(2)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E)
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)

input_window = 100 # number of input steps
output_window = 1 # number of prediction steps, in this model its fixed to one
block_len = input_window + output_window # for one input-output pair
batch_size = 10
train_size = 0.8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # div_term = torch.exp(
        #     torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        # )
        div_term = 1 / (10000 ** ((2 * np.arange(d_model)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term[0::2])
        pe[:, 1::2] = torch.cos(position * div_term[1::2])

        pe = pe.unsqueeze(0).transpose(0, 1) # [5000, 1, d_model],so need seq-len <= 5000
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        # print(self.pe[:x.size(0), :].repeat(1,x.shape[1],1).shape ,'---',x.shape)
        # dimension 1 maybe inequal batchsize
        return x + self.pe[:x.size(0), :].repeat(1,x.shape[1],1)


class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.input_embedding  = nn.Linear(1,feature_size)
        self.src_mask = None

        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        # src with shape (input_window, batch_len, 1)
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.input_embedding(src) # linear transformation before positional embedding
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
'''
In fact, assuming that the number of samples is N,
the length of the input sequence is m, and the backward prediction is k steps,
then length of a block [input : 1 , 2 ... m  -> output : k , k+1....m+k ]
should be (m+k) :  block_len, so to ensure that each block is complete,
the end element of the last block should be the end element of the entire sequence,
so the actual number of blocks is [N - block_len + 1]
'''
def create_inout_sequences(input_data, input_window ,output_window):
    inout_seq = []
    L = len(input_data)
    block_num =  L - block_len + 1
    # total of [N - block_len + 1] blocks
    # where block_len = input_window + output_window

    for i in range( block_num ):
        train_seq = input_data[i : i + input_window]
        #print(train_seq.shape)
        train_label = input_data[i + output_window : i + input_window + output_window]
        #print(train_label.shape)
        inout_seq.append((train_seq ,train_label))

    return torch.FloatTensor(np.array(inout_seq))

def get_data():
    # construct a littel toy dataset
    # time        = np.arange(0, 400, 0.1)
    # amplitude   = np.sin(time) + np.sin(time * 0.05) + \
    #               np.sin(time * 0.12) * np.random.normal(-0.2, 0.2, len(time))

    from sklearn.preprocessing import MinMaxScaler

    #loading weather data from a file
    from pandas import read_csv

# Read the CSV file into a DataFrame
# Read the CSV file into a DataFrame
    # Read the CSV file into a DataFrame
    df = df_plot['target']
    #df = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True)

    # Select the "Temp" column and squeeze it into a Series
    #series = df['Temp'].squeeze()

    # Convert the Series into a NumPy array
    numpy_array = df.values

    # Print the NumPy array format
    #print(numpy_array)

    # Squeeze the selected column into a Series
    #series = series.squeeze()
    # looks like normalizing input values curtial for the model
    scaler = MinMaxScaler(feature_range=(-1, 1))
    amplitude = scaler.fit_transform(numpy_array.reshape(-1, 1)).reshape(-1)
    #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    #print(numpy_array)
    #full_size = len(df)
    #train_size = int(0.3 * full_size)
    sampels = 10000 # use a parameter to control training size
    #print(sampels)
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]
    #print(train_data.shape)
    #print(test_data.shape)

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)

    train_sequence = create_inout_sequences( train_data,input_window ,output_window)
    '''
    train_sequence = train_sequence[:-output_window] # todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack..
    # looks like maybe solved
    '''
    #test_data = torch.FloatTensor(test_data).view(-1)
    test_data = create_inout_sequences(test_data,input_window,output_window)
    '''
    test_data = test_data[:-output_window] # todo: fix hack?
    '''
    # shape with (block , sql_len , 2 )
    return train_sequence.to(device),test_data.to(device)


def get_batch(input_data, i , batch_size):

    # batch_len = min(batch_size, len(input_data) - 1 - i) #  # Now len-1 is not necessary
    batch_len = min(batch_size, len(input_data) - i)
    data = input_data[ i:i + batch_len ]
    input = torch.stack([item[0] for item in data]).view((input_window,batch_len,1))
    # ( seq_len, batch, 1 ) , 1 is feature size
    target = torch.stack([item[1] for item in data]).view((input_window,batch_len,1))
    return input, target

def train(train_data):
    model.train() # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()
    all_losses = []  # List to store losses for each batch

    for batch, i in enumerate(range(0, len(train_data), batch_size)):  # Now len-1 is not necessary
        # data and target are the same shape with (input_window,batch_len,1)
        data, targets = get_batch(train_data, i , batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        all_losses.append(loss.item())  # Append current batch loss to the list

        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

    return all_losses

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    with torch.no_grad():
        # for i in range(0, len(data_source) - 1):
        for i in range(len(data_source)):  # Now len-1 is not necessary
            data, target = get_batch(data_source, i , 1) # one-step forecast
            output = eval_model(data)
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)

    #test_result = test_result.cpu().numpy() -> no need to detach stuff..
    len(test_result)

    plt.figure(figsize=(12, 6))
    plt.plot(test_result, color="red")
    plt.plot(truth[:12100], color="black", alpha=0.3, label='Actual')
    plt.ylim(-1, 1)
    plt.grid(True, which='both')
    plt.axhline(y=0, color='k')
    plt.savefig(folder_path_graph + 'transformer-epoch%d.png' % epoch)
    plt.show()
    plt.close()
    return total_loss / i


# predict the next n steps based on the input data
def predict_future(eval_model, data_source,steps, epoch):
    eval_model.eval()
    total_loss = 0.
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source , 0 , 1)
    with torch.no_grad():
        for i in range(0, steps):
            output = eval_model(data[-input_window:])
            # (seq-len , batch-size , features-num)
            # input : [ m,m+1,...,m+n ] -> [m+1,...,m+n+1]
            data = torch.cat((data, output[-1:])) # [m,m+1,..., m+n+1]

    data = data.cpu().view(-1)

    # I used this plot to visualize if the model pics up any long therm structure within the data.
    plt.figure(figsize=(12, 6))
    plt.plot(data, color="red")
    plt.plot(data[:input_window], color="black")
    plt.ylim(-1, 1)
    plt.grid(True, which='both')
    plt.axhline(y=0, color='k')
    plt.savefig(folder_path_graph + 'transformer-future%d.png' % epoch)
    plt.show()
    plt.close()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        # for i in range(0, len(data_source) - 1, eval_batch_size): # Now len-1 is not necessary
        for i in range(0, len(data_source), eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)
            total_loss += len(data[0]) * criterion(output, targets).cpu().item()
    return total_loss / len(data_source)

train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.0029
#lr = 0.0005
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

best_val_loss = float("inf")
epochs = 50 # The number of epochs
best_model = None

# Define an empty list to store learning rate values
# Define empty lists to store losses and learning rate values
train_losses = []
val_losses = []
learning_rates = []

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    if (epoch % 10 == 0):
        val_loss = plot_and_loss(model, val_data, epoch)
        predict_future(model, val_data, 150, epoch)
    else:
        val_loss = evaluate(model, val_data)

    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                                                                                   val_loss, math.exp(val_loss)))
    print('-' * 89)
    val_losses.append(val_loss)

    # Store training loss
    train_loss = evaluate(model, train_data)
    train_losses.append(train_loss)
    #print(train_loss)

    # Save learning rate
    current_lr = scheduler.get_last_lr()[0]
    learning_rates.append(current_lr)

    # if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step()

# Plot training and validation losses
plt.figure(figsize=(12, 6))
epochs_range = range(1, epochs + 1)
plt.plot(epochs_range, train_losses, label='Training Loss', color='blue')
plt.plot(epochs_range, val_losses, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()

# Plot learning rate
plt.twinx()
plt.plot(epochs_range, learning_rates, label='Learning Rate', color='green')
plt.ylabel('Learning Rate')

plt.grid(True)
plt.show()

In [ ]:
folder_path_model = "/content/drive/My Drive/Machine_Learning_24/Final_project/predict-energy-behavior-of-prosumers/timesieries_transformer/"
# Save the final model after training
final_model_path = folder_path_graph + 'final_transformer_model_100.pth'
torch.save(model.state_dict(), final_model_path)
print("Final model saved at:", final_model_path)


Final model saved at: /content/drive/My Drive/Machine_Learning_24/Final_project/predict-energy-behavior-of-prosumers/Graph_animation2/final_transformer_model_100.pth


# Make a animation

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import animation

# Folder path for the images
folder_path_graph = "/content/drive/My Drive/Machine_Learning_24/Final_project/predict-energy-behavior-of-prosumers/Graph_animation2/"

# Get a list of image files for epoch and future separately
epoch_image_files = sorted(glob.glob(folder_path_graph + 'transformer-epoch*.png'))
future_image_files = sorted(glob.glob(folder_path_graph + 'transformer-future*.png'))

# Initialize lists to store loaded images
epoch_images = []
future_images = []

# Load images for epoch
for image_file in epoch_image_files:
    img = mpimg.imread(image_file)
    epoch_images.append(img)

# Load images for future
for image_file in future_image_files:
    img = mpimg.imread(image_file)
    future_images.append(img)

# Function to animate the images
def animate(i, images):
    plt.clf()
    plt.imshow(images[i])
    plt.axis('off')  # Hide the axis

# Function to create animation from images
def create_animation(images, save_path, interval, fps):
    fig = plt.figure(figsize=(12, 6))
    ani = animation.FuncAnimation(fig, lambda i: animate(i, images), frames=len(images), interval=interval)
    ani.save(save_path, writer='ffmpeg', fps=fps)

# Create animations for epoch with slower speed
epoch_animation_path = os.path.join(folder_path_graph, 'epoch-animation_slow.mp4')
create_animation(epoch_images, epoch_animation_path, interval=1000, fps=5)  # Increased interval to 1000ms and decreased fps to 5

# Create animations for future with slower speed
future_animation_path = os.path.join(folder_path_graph, 'future-animation_slow.mp4')
create_animation(future_images, future_animation_path, interval=1000, fps=5)  # Increased interval to 1000ms and decreased fps to 5

print(f'Epoch animation saved to {epoch_animation_path}')
print(f'Future animation saved to {future_animation_path}')


# Calculation the MAE for the validation data with the transfomer model

In [ ]:
def calculate_mae(model, test_data):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients since we're not training
        # Initialize an empty list to store absolute errors
        absolute_errors = []

        # Iterate through the test data
        for i in range(len(test_data)):
            # Get input and target data for one sample
            data, target = get_batch(test_data, i, 1)

            # Use the model to make predictions
            output = model(data)

            # Convert predictions and targets to numpy arrays and flatten them
            predictions = output.squeeze().cpu().numpy()
            true_values = target.squeeze().cpu().numpy()

            # Calculate absolute errors for this sample
            absolute_errors.append(np.abs(predictions - true_values))

        # Compute the mean of absolute errors across all samples
        mae = np.mean(absolute_errors)

    return mae

# Load the test data
_, test_data = get_data()

# Calculate MAE using the trained model and the test data
mae = calculate_mae(model, test_data)

print("Mean Absolute Error (MAE) for the test dataset:", mae)


# Comparing the Transfomer with LGBM model
# Testing the LGBM model on a random split of validation and training *data*


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def get_data(df_plot):
    # Normalize the target variable
    #df_plot.drop(columns=['datetime'], inplace=True)
    df = df_plot['target']

    numpy_array = df.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    amplitude = scaler.fit_transform(numpy_array).reshape(-1)

    # Split the data into training and testing sets with a fixed size of 10000 samples
# Split the data into training and testing sets with a fixed size of 10,000 samples
    X_train, X_test, y_train, y_test = train_test_split(df_plot.drop(columns=['target']), amplitude, test_size=10000, random_state=42)


    return X_train, X_test, y_train, y_test

# Assuming df_plot is your DataFrame containing the data
def get_data(df_plot):
    # Normalize the target variable
    df_plot.drop(columns=['datetime'], inplace=True)
    df = df_plot['target']

    numpy_array = df.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    amplitude = scaler.fit_transform(numpy_array).reshape(-1)

    # Extract the first 10,000 entries as the testing set
    X_test = df_plot.drop(columns=['target']).iloc[:10000]
    y_test = amplitude[:10000]

    # Extract the remaining entries as the training set
    X_train = df_plot.drop(columns=['target']).iloc[10000:]
    y_train = amplitude[10000:]

    return X_train, X_test, y_train, y_test

def train_lgbm(X_train, y_train):
    # Define and train the LightGBM model
    lgb_model = lgb.LGBMRegressor()
    lgb_model.fit(X_train, y_train)

    return lgb_model

def calculate_mae_lgbm(model, X_test, y_test):
    # Make predictions using the LightGBM model
    predictions = model.predict(X_test)

    # Calculate the Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, predictions)

    return mae

# Normalize the target variable and split the data into training and testing sets
X_train, X_test, y_train, y_test = get_data(df_plot)

# Train the LightGBM model
lgbm_model = train_lgbm(X_train, y_train)

# Calculate MAE for the test dataset using the LightGBM model
mae_lgbm = calculate_mae_lgbm(lgbm_model, X_test, y_test)

print("Mean Absolute Error (MAE) for the test dataset (LightGBM):", mae_lgbm)


<ipython-input-81-a838ba435a2b>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10889
[LightGBM] [Info] Number of data points in the train set: 5247, number of used features: 47
[LightGBM] [Info] Start training from score -0.135893
Mean Absolute Error (MAE) for the test dataset (LightGBM): 0.21290364582946572


# Testing the LGBM model on a random split of validation and training *data*

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# Assuming df_plot is your DataFrame containing the data
def get_data(df_plot):
    # Normalize the target variable
    df_plot.drop(columns=['datetime'], inplace=True)
    df = df_plot['target']

    numpy_array = df.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    amplitude = scaler.fit_transform(numpy_array).reshape(-1)

    # Split the data into training and testing sets with a fixed size of 10000 samples
    X_train, X_test, y_train, y_test = train_test_split(df_plot.drop(columns=['target']), amplitude, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test


def train_lgbm(X_train, y_train):
    # Define and train the LightGBM model
    lgb_model = lgb.LGBMRegressor()
    lgb_model.fit(X_train, y_train)

    return lgb_model

def calculate_mae_lgbm(model, X_test, y_test):
    # Make predictions using the LightGBM model
    predictions = model.predict(X_test)

    # Calculate the Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, predictions)

    return mae

# Normalize the target variable and split the data into training and testing sets
X_train, X_test, y_train, y_test = get_data(df_plot)

# Train the LightGBM model
lgbm_model = train_lgbm(X_train, y_train)

# Calculate MAE for the test dataset using the LightGBM model
mae_lgbm = calculate_mae_lgbm(lgbm_model, X_test, y_test)

print("Mean Absolute Error (MAE) for the test dataset (LightGBM):", mae_lgbm)


<ipython-input-93-7a303abe143a>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008682 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11309
[LightGBM] [Info] Number of data points in the train set: 12197, number of used features: 47
[LightGBM] [Info] Start training from score -0.439473
Mean Absolute Error (MAE) for the test dataset (LightGBM): 0.04503654775846057


# Simple first test no split in production and consumption

- data witout revealed targets in the past

# Compare it to the other model n the same data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# Drop NaN values
df_clean = combined_df.dropna()

# Drop the date column
df_clean = df_clean.drop(columns=['datetime'])

# Split features and target
X = df_clean.drop(columns=['target'])
y = df_clean['target']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the models
model = LGBMRegressor()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the models using mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error for is_consumption=0: {mae}')



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.160809 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11947
[LightGBM] [Info] Number of data points in the train set: 1583585, number of used features: 52
[LightGBM] [Info] Start training from score 274.422218
Mean Absolute Error for is_consumption=0: 66.19094452598641


# Simple first test no split in production and consumption

- data with revealed targets in the past

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# Drop NaN values
df_clean = combined_df_copy.dropna()

# Drop the date column
df_clean = df_clean.drop(columns=['datetime'])

# Split features and target
X = df_clean.drop(columns=['target'])
y = df_clean['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the models
model = LGBMRegressor()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the models using mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error : {mae}')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.122167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13732
[LightGBM] [Info] Number of data points in the train set: 1580206, number of used features: 59
[LightGBM] [Info] Start training from score 274.472142
Mean Absolute Error for is_consumption=0: 38.994972136541215


# Test Split the input data in production and consumption

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# Assuming combined_df is already your DataFrame
# Drop NaN values
df_clean = combined_df_copy.dropna()

# Drop the date column
df_clean = df_clean.drop(columns=['datetime'])

# Separate the dataset based on is_consumption column
df_consumption_0 = df_clean[df_clean['is_consumption'] == 0]
df_consumption_1 = df_clean[df_clean['is_consumption'] == 1]

# Split features and target
X_0 = df_clean.drop(columns=['target'])
y_0 = df_clean['target']

X_1 = df_consumption_1.drop(columns=['target', 'is_consumption'])
y_1 = df_consumption_1['target']

In [ ]:
# Split the data into training and testing sets
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)


In [ ]:


# Split the data into training and testing sets
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)

# Initialize and train the models
model_0 = LGBMRegressor()
model_0.fit(X_train_0, y_train_0)

model_1 = LGBMRegressor()
model_1.fit(X_train_1, y_train_1)

# Make predictions
y_pred_0 = model_0.predict(X_test_0)
y_pred_1 = model_1.predict(X_test_1)

# Evaluate the models using mean_absolute_error
mae_0 = mean_absolute_error(y_test_0, y_pred_0)
mae_1 = mean_absolute_error(y_test_1, y_pred_1)

print(f'Mean Absolute Error for is_consumption=0: {mae_0}')
print(f'Mean Absolute Error for is_consumption=1: {mae_1}')


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.351300 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13732
[LightGBM] [Info] Number of data points in the train set: 1580206, number of used features: 59
[LightGBM] [Info] Start training from score 274.472142
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.222216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13726
[LightGBM] [Info] Number of data points in the train set: 790103, number of used features: 58
[LightGBM] [Info] Start training from score 459.357956
Mean Absolute Error for is_consumption=0: 38.99497213654133
Mean Absolute Error for is_consumption=1: 49.887957923733836


# Use of NN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Define a custom callback function to print train and validation loss
class PrintLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}/{self.params["epochs"]}, '
              f'Train Loss: {logs["loss"]:.4f}, '
              f'Validation Loss: {logs["val_loss"]:.4f}')

# Split the data into training and testing sets
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)

# Define a simple neural network model
model_0 = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_0.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

model_1 = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_1.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

# Compile the models
model_0.compile(optimizer='adam', loss='mean_squared_error')
model_1.compile(optimizer='adam', loss='mean_squared_error')

# Train the models with callbacks to print loss
print_loss_callback_0 = PrintLoss()
print_loss_callback_1 = PrintLoss()

model_0.fit(X_train_0, y_train_0, epochs=10, batch_size=32, verbose=0, validation_data=(X_test_0, y_test_0), callbacks=[print_loss_callback_0])
model_1.fit(X_train_1, y_train_1, epochs=10, batch_size=32, verbose=0, validation_data=(X_test_1, y_test_1), callbacks=[print_loss_callback_1])

# Make predictions
y_pred_0 = model_0.predict(X_test_0)
y_pred_1 = model_1.predict(X_test_1)

# Calculate mean absolute error
mae_0 = mean_absolute_error(y_test_0, y_pred_0)
mae_1 = mean_absolute_error(y_test_1, y_pred_1)

print(f'Mean Absolute Error for is_consumption=0: {mae_0}')
print(f'Mean Absolute Error for is_consumption=1: {mae_1}')


# Use onf LGBM Tuner
- Automated parameter optimisation
- only for target = 0


In [ ]:


import time
start_time = time.time()
# Initialize tuner for each set of input_dat  a
tuner = LGBMTuner(metric='rmspe', trials=50, refit=True, verbosity=1, visualization=True, seed=414243, device_type='cpu', grid='h,learning_rate')

# Define parameter ranges
tuner.grid['max_depth'] = (0, 100)
tuner.grid['learning_rate'] = (0.00001, 0.1)
tuner.grid['drop_rate'] = (0.05, 0.4)

# Fit tuner for each set of input_data
tuner.fit(X_train_0, y_train_0.squeeze())


# Fit optimized using the best hyperparameters
tuner.fit_optimized(X_train_0.to_numpy(), y_train_0.squeeze().to_numpy())

# Make predictions
y_pred_0 = tuner.predict(X_test_0)

# Calculate mean absolute error
mae_0 = mean_absolute_error(y_test_0, y_pred_0)

print(f'Mean Absolute Error for is_consumption=0: {mae_0}')

# Calculate the time taken to load the file
#time_taken_gpu = end_time - start_time
#print(time_taken_gpu)

 for target = 1


In [ ]:


import time
start_time = time.time()
# Initialize tuner for each set of input_dat  a
tuner = LGBMTuner(metric='rmspe', trials=50, refit=True, verbosity=1, visualization=True, seed=414243, device_type='cpu', grid='h,learning_rate')

# Define parameter ranges
tuner.grid['max_depth'] = (0, 100)
tuner.grid['learning_rate'] = (0.00001, 0.1)
tuner.grid['drop_rate'] = (0.05, 0.4)

# Fit tuner for each set of input_data
tuner.fit(X_train_1, y_train_1.squeeze())


# Fit optimized using the best hyperparameters
tuner.fit_optimized(X_train_1.to_numpy(), y_train_1.squeeze().to_numpy())

# Make predictions
y_pred_1 = tuner.predict(X_test_1)

# Calculate mean absolute error
mae_1 = mean_absolute_error(y_test_1, y_pred_1)

print(f'Mean Absolute Error for is_consumption=0: {mae_1}')

# Calculate the time taken to load the file
#time_taken_gpu = end_time - start_time

more trails

# Own Optuna test

In [ ]:
pip install lightgbm optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.8 MB/s eta 0:00:00


In [ ]:
import time
import optuna

In [ ]:
def objective(trial, X_train, y_train, X_test, y_test):
    param = {
        'objective': 'regression',
        'metric': 'l2',  # RMSPE not directly available, using l2 as a placeholder
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 0, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.00001, 0.1),
        'drop_rate': trial.suggest_float('drop_rate', 0.05, 0.4),
        'seed': 414243,
    }

    model = LGBMRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    return mae

start_time = time.time()

study_0 = optuna.create_study(direction='minimize')
study_0.optimize(lambda trial: objective(trial, X_train_0, y_train_0, X_test_0, y_test_0), n_trials=50)

study_1 = optuna.create_study(direction='minimize')
study_1.optimize(lambda trial: objective(trial, X_train_1, y_train_1, X_test_1, y_test_1), n_trials=50)

end_time = time.time()
time_taken = end_time - start_time

print(f"Time taken for tuning: {time_taken} seconds")

# Train final models with best hyperparameters
best_params_0 = study_0.best_params
best_params_0.update({'objective': 'regression', 'metric': 'l2', 'verbosity': -1, 'boosting_type': 'gbdt'})
model_0 = LGBMRegressor(**best_params_0)
model_0.fit(X_train_0, y_train_0)

best_params_1 = study_1.best_params
best_params_1.update({'objective': 'regression', 'metric': 'l2', 'verbosity': -1, 'boosting_type': 'gbdt'})
model_1 = LGBMRegressor(**best_params_1)
model_1.fit(X_train_1, y_train_1)

# Make predictions
y_pred_0 = model_0.predict(X_test_0)
y_pred_1 = model_1.predict(X_test_1)

# Evaluate the models using mean_absolute_error
mae_0 = mean_absolute_error(y_test_0, y_pred_0)
mae_1 = mean_absolute_error(y_test_1, y_pred_1)

print(f'Mean Absolute Error for is_consumption=0: {mae_0}')
print(f'Mean Absolute Error for is_consumption=1: {mae_1}')